In [ ]:
# coding=utf-8
# Copyright 2023 Frank Latos AC8P
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#
# Much appreciation to the Pymoo project for providing the optimization framework used herein:
#
# pymoo: Multi-objective Optimization in Python
# https://github.com/anyoptimization/pymoo
# https://pymoo.org/index.html
#


# A Class for Representing Yagis and Other Sectional Antennas:<br>Part 3, Setting Properties

You can define the geometry of your antenna by setting properties; here's a fuller discussion of the details.
<br />
<br />

The ```SolidAntenna``` class that you base your design on already has some default properties set; these are global properties that affect the overall antenna and all of its elements. As of this writing they look someting like:
```python
self.properties = {'tubing_dia':in2m([3/4, 5/8, 1/2]), 'section_lens':in2m([24,18]), 
                    'color':['lightblue','lightsteelblue','lightgray','whitesmoke'], 'nsides':9,
                    'boom':True, 'boomcolor':'steelblue', 
                    'boomext':in2m(4), 'boomy':in2m(3), 'boomz':in2m(2),
                    'rotate':[], 'translate':(0,0,0)}
```
You can add or modify these when you create a new object:
```python
my_antenna = SolidAntenna(properties={'translate':(0,0,20), 'display_mirror':True, 'seg_per_m':5}, n_elem=3)
```
or at any later time like this:
```python
my_antenna.properties.update({'translate':(0,0,30)})
```
<br />
<br />

Each element also has local properties. The local properties will override global properties for a single element. You set them like this:
```python
my_antenna[0] = {'len':in2m(108.75)}        # Set the length for element '0'
```
Note that dimensions are all in meters...in2m() is the inches --> meters conversion function.
<br />
<br />

### Creating a Yagi Antenna


<img src="img/Element_Initial_Pos_2.svg" alt="Drawing" style="width: 500px;"/>


<img src="img/Element_Initial_Pos_3.svg" alt="Drawing" style="width: 400px;"/>


The above diagram shows the initial position of a newly-created element: at the origin, extending along the positive y-axis. Move each element into its proper position using the ```'rotate'``` and ```'translate'``` properties.
<br /><br />
```python
my_antenna[0] = {'len':5.1, 'translate':(-1,0,0)}           # Reflector
my_antenna[1] = {'len':5.0}                                 # Driven
my_antenna[2] = {'len':4.9, 'translate':(1.2,0,0)}          # Director
```
Aim the antenna along the positive x-axis -- this corresponds to the 0&deg; direction on the azimuth chart<br />
Element [1] will be driven by default; you can override this with ```'driven':True```.
<br /><br />
For best performance create half-elements like these and use  ```GX 100 010``` or similar in your NEC code to reflect the elements across the x-z plane. Add ```'nec_mirror':True``` to the global properties to instead create additional ```GW``` cards for the other element halves (and don't use ```GX```, of course). In either case use ```'display_mirror':True``` to create the mirrored elements in the solid model.
<br /><br />
Now use the global ```'rotate'``` and ```'translate'``` properties to move the antenna away from the origin if desired. In all other cases, local properties simply override global ones, but in the case of these two, both sets can be used independently. For example, rotate to a vertical orientation, aim the antenna 10&deg; above the horizon, rotate to a compass bearing of 45&deg;, and position at a height of 20 meters:
```python
my_ant = SolidAntenna(properties={'rotate':[('x',90), ('y',-10), ('z',-45)], 'translate':(0,0,20), ....
```
The arrows in the diagram show the direction of positive rotation for each axis.



In [2]:

from necutil import in2m, SolidAntenna, SolidObjVisualizer,nec5_sim_stdio3,plot_azimuth,plot_elevation

three_el_yagi = SolidAntenna(properties={'rotate':[('x',90), ('y',-10), ('z',-45)], 'translate':(0,0,10), 'display_mirror':True}, n_elem=3)

three_el_yagi[0] = {'len':5.1, 'translate':(-1,0,0)}           # Reflector
three_el_yagi[1] = {'len':5.0}                                 # Driven
three_el_yagi[2] = {'len':4.9, 'translate':(1.2,0,0)}          # Director

v = SolidObjVisualizer(x=(-10,10), y=(-10,10), z=(0,21))        # Sets the limits of displayed volume
v.add(three_el_yagi)
v.show()


<img src="img/3el_yagi_2.png" alt="Drawing" style="width: 400px;"/>

<br /><br />
### Property Summary
<br /><br />

| Property | |
| ----- | ----- |
| tubing_dia | Diameters of tubing sections:  ```'tubing_dia':in2m([1, 7/8, 3/4, 5/8])``` |
| display_dia | Optionally specify different diameters for displayed tubing |
| section_lens | Fixed lengths of inner tubing sections: ```'section_lens':in2m([24,18])``` |
| color | Display color for sections (standard CSS color names): <br />```'color':'blue'``` or ```'color':['lightblue','lightsteelblue','lightgray','whitesmoke']```|
| nsides | Number of sides used in polygonal approximation of circular tubing in displayed model|
| | |
| boom | True = creates a nonfunctional rectangular boom for Yagi in displayed model|
| boom_ext | Distance boom extends beyond first, last elements|
| boomy, boomz | Boom cross-section dimensions|
| boomcolor | Boom display color:  ```'boomcolor':'steelblue'```|
| | |
| rotate | List of rotations (degrees), performed in order:  ```'rotate':[('x',90),('y',-10),('z',-45)]```|
| translate | Move elements(s):  ```'translate':(x,y,z)```|
| | |
| len | Overall element length = variable outer section + fixed-length inner sections|
| tag | Integer tag used by NEC; default = 1 for element[0], etc.|
| driven | Identify the driven element; default = element[1];   ```'driven':True```|
| | |
| segnums | Number of NEC segments for each section: ```'segnums':[10,8,12]```|
| seg_per_m | Alternately, specify segments per meter of section length: ```'seg_per_m':5```|
| | |
| text | Text displayed when mouse cursor hovers: ```'text':'Reflector'```|
| suffix | Optional suffix when hovering over element section: ```'suffix':['7/8', '3/4', '5/8']```|
| | |
| nec_mirror | Generate ```GW``` cards in NEC output for mirrored elements  ```'nec_mirror':True```|
| display_mirror | Display mirrored elements  ```'display_mirror':True```|





